In [1]:
import pandas as pd
df_movies = pd.read_csv('final_movies.csv')
df_genres = pd.read_csv('movies_100k_genres.csv')
col_names_for_user_ratings = ['user_id', 'movie_id', 'rating', 'timestamp']
col_names_for_users = ['user_id' , 'age' , 'gender' , 'occupation' ,'zip code']
col_names_for_user_ratings = ['user_id', 'movie_id', 'rating', 'timestamp']
df_user_ratings = pd.read_csv(
    'ml-100k/u.data',
    sep='\t',
    header=None,
    names=col_names_for_user_ratings,
    encoding='latin-1'
)

# Step 1: Count interactions
movie_interactions = (
    df_user_ratings
    .groupby('movie_id')
    .size()
    .reset_index(name='interaction_count')
    .sort_values(by='interaction_count', ascending=False)
)

# Step 2: Attach movie titles
movie_with_titles = movie_interactions.merge(df_movies[['movie_id', 'title']], on='movie_id', how='left')

# Step 3: Show results
print(movie_with_titles.head(10))


df_users = pd.read_csv(
    'ml-100k/u.user',
    sep='|',
    header=None,
    names=col_names_for_users,
    encoding='latin-1'
)


   movie_id  interaction_count                          title
0        50                583               Star Wars (1977)
1       258                509                 Contact (1997)
2       100                508                   Fargo (1996)
3       181                507      Return of the Jedi (1983)
4       294                485               Liar Liar (1997)
5       286                481    English Patient, The (1996)
6       288                478                  Scream (1996)
7         1                452               Toy Story (1995)
8       300                431           Air Force One (1997)
9       121                429  Independence Day (ID4) (1996)


In [2]:
df_movies.head(100)

,movie_id,title,release_year,imdbId,tmdbId,imdb_rating,adult,budget,origin_country,vote_count,...,tmdb_popularity_score,production_companies,production_companies_origin_country,runtime,revenue,spoken_languages,directors_info,popular,unique_producers,popular_producer
0,1,Toy Story (1995),1995,114709,862,8.3,False,30000000,US,18849,...,23.1155,Pixar,US,81,394436586.0,English,"[['John Lasseter', '2.0842', 'Male']]",1,['Pixar'],0
1,2,GoldenEye (1995),1995,113189,710,7.2,False,60000000,GB,4012,...,6.4216,EON Productions,GB,130,352194034.0,"English,Pусский,Español","[['Martin Campbell', '0.9017', 'Male']]",1,['EON Productions'],0
2,3,Four Rooms (1995),1995,113101,5,6.7,False,4000000,US,2694,...,3.4017,"Miramax,A Band Apart","US,US",98,4257354.0,English,"[['Allison Anders', '0.3886', 'Female'], ['Ale...",0,"['A Band Apart', 'Miramax']",1
3,4,Get Shorty (1995),1995,113161,8012,6.9,False,30250000,US,1095,...,4.0703,"Metro-Goldwyn-Mayer,Jersey Films","US,US",105,115101622.0,English,"[['Barry Sonnenfeld', '0.7884', 'Male']]",0,"['Jersey Films', 'Metro-Goldwyn-Mayer']",1
4,5,Copycat (1995),1995,112722,1710,6.6,False,20000000,US,1014,...,4.3406,"New Regency Pictures,Regency Enterprises,Warne...","US,US,US",124,32000000.0,"English,Deutsch,Italiano","[['Jon Amiel', '0.4666', 'Male']]",0,"['Warner Bros. Pictures', 'Regency Enterprises...",1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,110,Operation Dumbo Drop (1995),1995,114048,27281,5.2,False,0,US,181,...,1.3180,"Interscope Communications,Walt Disney Pictures...","US,US,US",107,0.0,"English,Español,Tiếng Việt","[['Simon Wincer', '0.5484', 'Male']]",0,"['Interscope Communications', 'Walt Disney Pic...",1
96,111,"Truth About Cats & Dogs, The (1996)",1996,117979,8866,6.3,False,0,US,324,...,0.8527,"Noon Attack Pictures,20th Century Fox","US,US",93,3407.0,English,"[['Michael Lehmann', '0.8774', 'Male']]",0,"['Noon Attack Pictures', '20th Century Fox']",1
97,112,Flipper (1996),1996,116322,36355,5.3,False,25530000,US,321,...,1.5222,"The Bubble Factory,Universal Pictures,American...","US,US,",95,20080020.0,"Français,English","[['Alan Shapiro', '0.2599', 'Male']]",0,"['The Bubble Factory', 'American Films', 'Univ...",1
98,113,"Horseman on the Roof, The (Hussard sur le toit...",1996,113362,11876,7.0,False,35000000,FR,142,...,0.8914,"Hachette Première,France 2 Cinéma,Canal+","FR,FR,FR",135,15000000.0,"Français,Italiano","[['Jean-Paul Rappeneau', '0.2048', 'Male']]",0,"['France 2 Cinéma', 'Hachette Première', 'Cana...",0


In [3]:
unique_user_count = df_user_ratings['user_id'].nunique()
print(unique_user_count)

943


In [4]:

df_user_ratings["date"] = pd.to_datetime(df_user_ratings["timestamp"], unit='s')

# Count interactions per user
interaction_counts = df_user_ratings.groupby('user_id').size()

# Filter users with more than 60 interactions (strictly > 100)
valid_users = interaction_counts[interaction_counts >= 100].index

# Apply the filter
df_filtered_user_ratings = df_user_ratings[df_user_ratings['user_id'].isin(valid_users)].copy()

# Sort interactions chronologically per user
df_filtered_user_ratings.sort_values(['user_id', 'date'], inplace=True)

# Split each user's history: 80% train, 20% test
train_list = []
test_list = []

for uid, user_df in df_filtered_user_ratings.groupby('user_id', sort=False):
    n = len(user_df)
    split_pt = int(n * 0.8)
    train_list.append(user_df.iloc[:split_pt])
    test_list.append(user_df.iloc[split_pt:])

# Combine all user splits
train_df = pd.concat(train_list).reset_index(drop=True)
test_df = pd.concat(test_list).reset_index(drop=True)

# Output some stats
print(f"After filtering, {len(valid_users)} users remain.")
print(f"Train set: {len(train_df)} rows")
print(f"Test set:  {len(test_df)} rows")

print(f"Minimum number of interactions among kept users: {interaction_counts[valid_users].min()}")
print(f"Number of users with <= 100 interactions: {(interaction_counts < 100).sum()}")


After filtering, 364 users remain.
Train set: 59469 rows
Test set:  15053 rows
Minimum number of interactions among kept users: 100
Number of users with <= 100 interactions: 579


pick random user for few shot examples

In [5]:
import random

# Turn your valid_users (Index or list) into a plain list:
valid_users_list = list(valid_users)


three_picked = [1,92,433]
print("Picked users:", three_picked)
remaining_valid_users_list = [u for u in valid_users_list if u not in three_picked]
print("Remaining valid users:", remaining_valid_users_list)

Picked users: [1, 92, 433]
Remaining valid users: [5, 6, 7, 10, 11, 13, 15, 16, 18, 21, 22, 23, 26, 38, 42, 43, 44, 49, 56, 57, 58, 59, 60, 62, 64, 70, 72, 82, 83, 85, 87, 90, 94, 95, 99, 102, 104, 109, 110, 116, 119, 125, 128, 130, 141, 144, 145, 151, 152, 158, 159, 160, 174, 177, 178, 181, 184, 188, 189, 193, 194, 197, 198, 200, 201, 207, 210, 213, 214, 216, 221, 222, 223, 224, 230, 233, 234, 236, 239, 244, 246, 249, 250, 254, 256, 262, 263, 264, 267, 268, 269, 270, 271, 276, 279, 280, 286, 290, 291, 292, 293, 294, 295, 296, 297, 298, 299, 301, 303, 305, 307, 308, 311, 312, 313, 314, 318, 320, 321, 325, 326, 327, 328, 330, 332, 334, 336, 339, 342, 343, 344, 345, 346, 347, 354, 360, 361, 363, 373, 374, 378, 379, 380, 381, 385, 387, 389, 391, 392, 393, 394, 397, 398, 399, 401, 405, 406, 407, 409, 416, 417, 425, 426, 429, 435, 436, 437, 442, 445, 447, 450, 452, 453, 454, 455, 456, 457, 458, 459, 463, 466, 468, 472, 474, 478, 479, 484, 486, 487, 488, 489, 493, 495, 496, 497, 498, 499, 50

selection of sample

In [6]:
import numpy as np

# --- 7) Sampling helper ---
def select_sample(user_ids, k=10, random_state=42):
    """
    For each user:
      - ground_truth_total_itemIds: all items (train+test)
      - ground_truth_test_itemIds: only test items
      - sample_random: up to k random items from their train history, ordered chronologically
    """
    rng = np.random.default_rng(random_state)
    records = []

    for uid in user_ids:
        user_all = df_filtered_user_ratings.loc[df_filtered_user_ratings.user_id == uid]
        total_items = user_all['movie_id'].tolist()

        user_test = test_df.loc[test_df.user_id == uid]
        test_items = user_test['movie_id'].tolist()

        user_train = train_df.loc[train_df.user_id == uid].copy()

        if user_train.empty:
            # No train data — keep sample empty but still record ground truth
            random_sample = []
        else:
            # sample up to k rows (no replacement)
            n_pick = min(k, len(user_train))
            # Use pandas sample with a deterministic seed per user for reproducibility
            # Seed is derived from (random_state, uid) so different users differ but are reproducible
            seed = (hash((random_state, int(uid))) % (2**32 - 1))
            sample_df = (
                user_train
                .sample(n=n_pick, replace=False, random_state=seed)
                .sort_values('date', ascending=True)
            )
            # keep only movie_id and rating
            keep_cols = [c for c in ['movie_id', 'rating'] if c in sample_df.columns]
            random_sample = sample_df[keep_cols].values.tolist()

        records.append({
            'user_id': uid,
            'ground_truth_total_itemIds': total_items,
            'ground_truth_test_itemIds': test_items,
            'sample_random': random_sample
        })

    return pd.DataFrame(records)

# --- 8) Build the two DataFrames ---
df_filtered_user_data = select_sample(remaining_valid_users_list)
df_filtered_example_user_data = select_sample(three_picked)

print("df_filtered_user_data shape:", df_filtered_user_data.shape)
print("df_filtered_example_user_data shape:", df_filtered_example_user_data.shape)

df_filtered_user_data shape: (362, 4)
df_filtered_example_user_data shape: (3, 4)


In [ ]:
# -*- coding: utf-8 -*-
import os, re, time, json, random
from pathlib import Path
from concurrent.futures import ThreadPoolExecutor, as_completed

import pandas as pd
from tqdm import tqdm
from mistralai import Mistral

# =========================
# CONFIG
# =========================
API_KEY = ""  # <-- fill this
#MODEL_NAME = "open-mistral-7b"
MODEL_NAME="mistral-large-2407"
client = Mistral(api_key=API_KEY)

TOP_K = 10
NORMAL_TEMPS = [0.0]           # normal run (one file)
SC_TEMPS = [0.2, 0.5, 0.7, 1.0]     # self-consistency temperatures
MAX_WORKERS = 2           # good balance for a 24-core/120GB machine
PAUSE_SECONDS = 1.2

OUTPUT_NORMAL = Path("mistral-2407/results_normal_revised")
OUTPUT_SC = Path("mistral-2407/results_sc")
OUTPUT_NORMAL.mkdir(parents=True, exist_ok=True)
OUTPUT_SC.mkdir(parents=True, exist_ok=True)

# =========================
# DATA ASSUMPTIONS
# Must be defined BEFORE running this script:
#   df_movies:              ['movie_id', 'title']
#   df_genres:              'movie_id' + one-hot genre columns
#   df_filtered_user_data:  ['user_id', 'sample_random' (list[(movie_id, rating)])]
# =========================
assert "movie_id" in df_genres.columns, "df_genres must have 'movie_id'"
assert {"movie_id", "title"}.issubset(df_movies.columns), "df_movies needs 'movie_id','title'"
assert "user_id" in df_filtered_user_data.columns, "df_filtered_user_data must have 'user_id'"

# Lookups
# Lookups
genre_cols = [c for c in df_genres.columns if c != "movie_id"]
movie_id_to_genres = {
    int(row.movie_id): [g for g in genre_cols if int(row[g]) == 1]
    for _, row in df_genres.iterrows()
}
id_to_title = dict(zip(df_movies.movie_id, df_movies.title))
title_to_id = dict(zip(df_movies.title.str.strip(), df_movies.movie_id))  # <-- ensure we have this

# =========================
# FAIRNESS PROMPTS
# =========================
FAIRNESS_PROMPTS = {
    "neutral": "",
}
FAIRNESS_LEVELS = ['neutral']

# =========================
# EXAMPLES (few-shot) loader
# expects files like: examples/examples_fair2.json with structure:
# { "random": [ { "user_history": [... or string ...], "recommendation": "1. X, 2. Y, ..." }, ... ] }
# =========================
def load_examples():
    all_ex = {}
    for lvl in FAIRNESS_LEVELS:
        fp = Path(f"examples/examples_{lvl}.json")
        all_ex[lvl] = json.load(fp.open(encoding="utf-8")) if fp.exists() else {}
    return all_ex

EX_BY_FAIR = load_examples()

# =========================
# TEMPLATE SETS
# A: JSON with ranks   | B: JSON titles only   | C: Plain-text numbered list
# (system/user split; {history} only used in user; others via system)
# =========================
PROMPT_TEMPLATES_A = {
    "zero_shot": {
        "system": (
            "You are a helpful movie recommender.\n"
            "{fairness}Recommend exactly the top-{k} ranked movies.\n"
            "RULES (STRICT):\n"
            "- Recommend {k} distinct movies not present in the watched list below.\n"
            "- No duplicates in recommendations.\n"
            "- Output a single JSON object and nothing else.\n"
            "SCHEMA (STRICT):\n"
            "{{\"k\": {k}, \"recommendations\": [{\"rank\": 1, \"title\": \"<Movie 1>\"}, ..., "
            "{\"rank\": {k}, \"title\": \"<Movie {k}>\"}]}}\n"
        ),
        "user": "User's watched movies (exclude all of these in recommendation):\n{history}"
    },
    "few_shot": {
        "system": (
            "You are a helpful movie recommender.\n"
            "Below are demonstrated examples (no reasoning shown). Each example shows a user's history and a FINAL_JSON object that follows the strict schema.\n"
            "{fairness}Recommend exactly the top-{k} ranked movies.\n"
            "RULES (STRICT):\n"
            "- Recommend {k} distinct movies not present in the watched list below.\n"
            "- No duplicates in recommendations.\n"
            "- Output a single JSON object and nothing else.\n"
            "SCHEMA (STRICT):\n"
            "{{\"k\": {k}, \"recommendations\": [{\"rank\": 1, \"title\": \"<Movie 1>\"}, ..., "
            "{\"rank\": {k}, \"title\": \"<Movie {k}>\"}]}}\n"
        ),
        "user": "User's watched movies (exclude all of these in recommendation):\n{history}"
    },
    "zero_shot_cot": {
        "system": (
            "You are a helpful movie recommender.\n"
            "{fairness}Recommend exactly the top-{k} ranked movies.\n"
            "RULES (STRICT):\n"
            "- Recommend {k} distinct movies not present in the watched list below.\n"
            "- No duplicates in recommendations.\n"
            "SCHEMA (STRICT):\n"
            "{{\"k\": {k}, \"recommendations\": [{\"rank\": 1, \"title\": \"<Movie 1>\"}, ..., "
            "{\"rank\": {k}, \"title\": \"<Movie {k}>\"}]}}\n"
            "Before answering, LET'S THINK STEP BY STEP.\n"

        ),
        "user": "User's watched movies (exclude all of these in recommendation):\n{history}"
    },
    "few_shot_cot": {
        "system": (
            "You are a helpful movie recommender.\n"
            "Below are demonstrated examples (no reasoning shown). Each example shows a user's history and a FINAL_JSON object that follows the strict schema.\n"
            "{fairness}Recommend exactly the top-{k} ranked movies.\n"
            "RULES (STRICT):\n"
            "- Recommend {k} distinct movies not present in the watched list below.\n"
            "- No duplicates in recommendations.\n"
            "SCHEMA (STRICT):\n"
            "{{\"k\": {k}, \"recommendations\": [{\"rank\": 1, \"title\": \"<Movie 1>\"}, ..., "
            "{\"rank\": {k}, \"title\": \"<Movie {k}>\"}]}}\n"
            "Before answering, LET'S THINK STEP BY STEP.\n"
        ),
        "user": "User's watched movies (exclude all of these in recommendation):\n{history}"
    },
}

PROMPT_TEMPLATES_B = {
    "zero_shot": {
        "system": (
            "You are a helpful movie recommender.\n"
            "{fairness}Recommend exactly top-{k} movies.\n"
            "RULES (STRICT):\n"
            "- Recommend {k} distinct movies not present in the watched list below.\n"
            "- No duplicates.\n"
            "- Output a single JSON object and nothing else.\n"
            "SCHEMA (STRICT):\n"
            '{{\"k\": {k}, \"recommendations\": [\"<Movie 1>\", \"<Movie 2>\", ..., \"<Movie {k}>\"]}}'
        ),
        "user": "User's watched movies (exclude all of these in recommendation):\n{history}"
    },
    "few_shot": {
        "system": (
            "You are a helpful movie recommender.\n"
            "Below are demonstrated examples (no reasoning shown). Each example shows a user's history and a FINAL_JSON object that follows the strict schema.\n"
            "RULES (STRICT):\n"
            "- Recommend {k} distinct movies not present in the watched list below.\n"
            "- No duplicates.\n"
            "- Output a single JSON object and nothing else.\n"
            "SCHEMA (STRICT):\n"
            '{{\"k\": {k}, \"recommendations\": [\"<Movie 1>\", \"<Movie 2>\", ..., \"<Movie {k}>\"]}}'
        ),
        "user": "User's watched movies (exclude all of these in recommendation):\n{history}"
    },
    "zero_shot_cot": {
        "system": (
            "You are a helpful movie recommender.\n"
            "{fairness}Recommend exactly top-{k} movies.\n"
            "RULES (STRICT):\n"
            "- Recommend {k} distinct movies not present in the watched list below.\n"
            "- No duplicates.\n"
            "SCHEMA (STRICT):\n"
            '{{\"k\": {k}, \"recommendations\": [\"<Movie 1>\", \"<Movie 2>\", ..., \"<Movie {k}>\"]}}\n'
            "Before answering, LET'S THINK STEP BY STEP.\n"
        ),
        "user": "User's watched movies (exclude all of these in recommendation):\n{history}"
    },
    "few_shot_cot": {
        "system": (
            "You are a helpful movie recommender.\n"
            "Below are demonstrated examples (no reasoning shown). Each example shows a user's history and a FINAL_JSON object that follows the strict schema.\n"
            "{fairness}Recommend exactly top-{k} movies.\n"
            "RULES (STRICT):\n"
            "- Recommend {k} distinct movies not present in the watched list below.\n"
            "- No duplicates.\n"
            "SCHEMA (STRICT):\n"
            '{{\"k\": {k}, \"recommendations\": [\"<Movie 1>\", \"<Movie 2>\", ..., \"<Movie {k}>\"]}}\n'
            "Before answering, LET'S THINK STEP BY STEP.\n"
        ),
        "user": "User's watched movies (exclude all of these in recommendation):\n{history}"
    },
}

PROMPT_TEMPLATES_C = {
    "zero_shot": {
        "system": (
            "You are a helpful movie recommender.\n"
            "{fairness}Recommend exactly top-{k} movies.\n"
            "RULES (STRICT):\n"
            "- Recommend {k} distinct movies not present in the watched list below.\n"
            "- No duplicates.\n"
            "- Output EXACTLY this plain-text format (no extra text):\n"
            "RECOMMENDATIONS:\n1) <Movie 1>\n2) <Movie 2>\n...\n{k}) <Movie {k}>"
        ),
        "user": "User's watched movies (exclude all of these in recommendation):\n{history}"
    },
    "few_shot": {
        "system": (
            "You are a helpful movie recommender.\n"
            "Below are demonstrated examples (no reasoning shown). Each example shows a user's history and the response.\n"
            "{fairness}Recommend exactly top-{k} movies.\n"
            "RULES (STRICT):\n"
            "- Recommend {k} distinct movies not present in the watched list below.\n"
            "- No duplicates.\n"
            "- Output EXACTLY this plain-text format (no extra text):\n"
            "RECOMMENDATIONS:\n1) <Movie 1>\n2) <Movie 2>\n...\n{k}) <Movie {k}>"
        ),
        "user": "User's watched movies (exclude all of these in recommendation):\n{history}"
    },
    "zero_shot_cot": {
        "system": (
            "You are a helpful movie recommender.\n"
            "{fairness}Recommend exactly top-{k} movies.\n"
            "RULES (STRICT):\n"
            "- Recommend {k} distinct movies not present in the watched list below.\n"
            "- No duplicates.\n"
            "- Follow output format:\n"
            "RECOMMENDATIONS:\n1) <Movie 1>\n2) <Movie 2>\n...\n{k}) <Movie {k}>\n"
            "Before answering, LET'S THINK STEP BY STEP.\n"
        ),
        "user": "User's watched movies (exclude all of these in recommendation):\n{history}"
    },
    "few_shot_cot": {
        "system": (
            "You are a helpful movie recommender.\n"
            "Below are demonstrated examples (no reasoning shown). Each example shows a user's history and the response.\n"
            "RULES (STRICT):\n"
            "- Recommend {k} distinct movies not present in the watched list below.\n"
            "- No duplicates.\n"
            "- Follow output format:\n"
            "RECOMMENDATIONS:\n1) <Movie 1>\n2) <Movie 2>\n...\n{k}) <Movie {k}>\n"
            "Before answering, LET'S THINK STEP BY STEP.\n"
        ),
        "user": "User's watched movies (exclude all of these in recommendation):\n{history}"
    },
}

TEMPLATE_SETS = {"A": PROMPT_TEMPLATES_A, "B": PROMPT_TEMPLATES_B} if False else {"A": PROMPT_TEMPLATES_A, "B": PROMPT_TEMPLATES_B, "C": PROMPT_TEMPLATES_C}

def _clean_title(s: str) -> str:
    """
    Normalize example titles:
    - coerce to str
    - strip whitespace
    - drop a trailing comma (e.g., 'Copycat (1995),')
    """
    return re.sub(r'[,\s]+$', '', str(s).strip())

def _format_example_history_from_titles(titles, default_rating=3):
    """
    Build a numbered history (same style as real prompts) from a list of titles.
    Example output:
      1. "Toy Story (1995)" – Animation|Children's|Comedy (rating 3)
      2. "Rumble in the Bronx (1995)" – Action|Adventure|Crime (rating 3)
    """
    if not titles:
        return ""

    lines = []
    for idx, raw in enumerate(titles, start=1):
        t = _clean_title(raw)
        mid = title_to_id.get(t)
        if mid is not None:
            genres = movie_id_to_genres.get(int(mid), [])
            gtxt = "|".join(genres) if genres else "Unknown"
        else:
            # Title not found in df_movies (spelling/variant) → still render gracefully
            gtxt = "Unknown"
        lines.append(f'{idx}. "{t}" – {gtxt} (rating {default_rating})')
    return "\n".join(lines)

def _history_text_from_example(hist):
    """
    Previously this returned a bullet list. Now it returns the SAME formatted
    numbered history used for real prompts, with genres + a default rating.
    - If 'hist' is a list of titles, format using df lookups.
    - If 'hist' is a string, try to split by lines/commas; otherwise render as-is.
    """
    if isinstance(hist, list):
        return "User's watched movies (exclude all of these in recommendation):\n" + _format_example_history_from_titles(hist)
    # If string, try to split on newlines, else commas → then format
    s = str(hist or "").strip()
    if not s:
        return "User's watched movies (exclude all of these in recommendation):\n"
    # try newline first, fallback to commas
    parts = [p for p in re.split(r"\n+", s) if p.strip()] or [p for p in s.split(",") if p.strip()]
    return "User's watched movies (exclude all of these in recommendation):\n" + _format_example_history_from_titles(parts)

_REC_ITEM_RE = re.compile(r'(\d+)\.\s*(.+?)(?=(?:\s*,\s*\d+\.\s*)|$)')

def _ranked_obj_from_titles(titles, k_default=10):
    titles = [str(t).strip() for t in titles if str(t).strip()]
    items = [{"rank": i, "title": t} for i, t in enumerate(titles[:k_default], start=1)]
    if not items:
        items = [{"rank": i, "title": f"<Movie {i}>"} for i in range(1, k_default + 1)]
    return {"k": len(items), "recommendations": items}

def recommendation_line_to_json_obj(rec_any, k_default: int = 10):
    if isinstance(rec_any, dict) and "recommendations" in rec_any:
        items = []
        for i, it in enumerate(rec_any.get("recommendations", []), start=1):
            if isinstance(it, dict):
                title = it.get("title") or it.get("name") or str(it)
                rank = int(it.get("rank", i))
            else:
                title = str(it)
                rank = i
            items.append({"rank": int(rank), "title": str(title)})
        items = sorted(items, key=lambda x: x["rank"])[:k_default]
        if not items:
            items = [{"rank": i, "title": f"<Movie {i}>"} for i in range(1, k_default + 1)]
        return {"k": len(items), "recommendations": items}

    if isinstance(rec_any, list):
        titles = []
        for i, it in enumerate(rec_any, start=1):
            if isinstance(it, dict):
                t = it.get("title") or it.get("name") or str(it)
            else:
                t = str(it)
            t = t.strip()
            if t:
                titles.append(t)
        return _ranked_obj_from_titles(titles, k_default=k_default)

    rec_text = (rec_any or "").strip()
    if rec_text.startswith("{"):
        try:
            obj = json.loads(rec_text)
            if isinstance(obj, dict) and "recommendations" in obj:
                return recommendation_line_to_json_obj(obj, k_default=k_default)
        except Exception:
            pass

    items = []
    for m in _REC_ITEM_RE.finditer(rec_text):
        try:
            rank = int(m.group(1))
        except Exception:
            continue
        title = m.group(2).strip().rstrip(", ").replace(" …", "").replace("...", "").strip()
        items.append({"rank": rank, "title": title})

    if not items:
        if "," in rec_text:
            titles = [t.strip() for t in rec_text.split(",") if t.strip()]
            return _ranked_obj_from_titles(titles, k_default=k_default)
        return _ranked_obj_from_titles([], k_default=k_default)

    items = sorted(items, key=lambda x: x["rank"])[:k_default]
    return {"k": len(items), "recommendations": items}

def _assistant_text_for_set_A(rec_any, k_default=10):
    obj = recommendation_line_to_json_obj(rec_any, k_default=k_default)
    return json_compact(obj)

def _assistant_text_for_set_B(rec_any, k_default=10):
    objA = recommendation_line_to_json_obj(rec_any, k_default=k_default)
    titles = [r["title"] for r in objA.get("recommendations", [])]
    return json_compact({"k": len(titles), "recommendations": titles})

def _assistant_text_for_set_C(rec_any, k_default=10):
    objA = recommendation_line_to_json_obj(rec_any, k_default=k_default)
    titles = [r["title"] for r in objA.get("recommendations", [])]
    lines = ["RECOMMENDATIONS:"] + [f"{i}) {t}" for i, t in enumerate(titles, start=1)]
    return "\n".join(lines)


def json_compact(obj) -> str:
    return json.dumps(obj, ensure_ascii=False, separators=(",", ":"))


def build_example_turns(set_name: str, fairness: str, k_default=10, max_examples=2, strategy="random"):
    ex_list = EX_BY_FAIR.get(fairness, {}).get(strategy, [])
    if not ex_list:
        return []
    turns = []
    for ex in ex_list[:max_examples]:
        u_hist = _history_text_from_example(ex.get("user_history", ""))
        if set_name == "A":
            a_out = _assistant_text_for_set_A(ex.get("recommendation", ""), k_default)
        elif set_name == "B":
            a_out = _assistant_text_for_set_B(ex.get("recommendation", ""), k_default)
        else:
            a_out = _assistant_text_for_set_C(ex.get("recommendation", ""), k_default)
        turns.append({"role": "user", "content": u_hist})
        turns.append({"role": "assistant", "content": a_out})
    return turns

# =========================
# BUILD MESSAGES (system + few-shot dialogue turns + real user)
# =========================
def render(text: str, **vals) -> str:
    for k, v in vals.items():
        text = text.replace("{"+k+"}", str(v))
    return text

def format_history(pairs):
    if not pairs: return ""
    lines = []
    for i, (mid, rating) in enumerate(pairs, start=1):
        title = id_to_title.get(mid, f"Unknown({mid})")
        genres = movie_id_to_genres.get(mid, [])
        gtxt = "|".join(genres) if genres else "Unknown"
        rtxt = rating if pd.notna(rating) else "N/A"
        lines.append(f'{i}. "{title}" – {gtxt} (rating {rtxt})')
    return "\n".join(lines)

def build_messages(set_name, style, fairness_key, k, real_history_text):
    fairness_text = FAIRNESS_PROMPTS.get(fairness_key, "")
    tpl = TEMPLATE_SETS[set_name][style]

    messages = [{"role": "system", "content": render(tpl["system"], fairness=fairness_text, k=k)}]

    if "few_shot" in style:
        messages.extend(build_example_turns(set_name, fairness_key, k_default=k, max_examples=2, strategy="random"))

    user_text = render(tpl["user"], history=real_history_text, k=k)
    messages.append({"role": "user", "content": user_text})

    print("\n=== BUILT MESSAGES ===")
    for m in messages:
        role = m["role"].upper()
        print(f"[{role}]\n{m['content']}\n")
    print("======================\n")

    return messages

def chat_once(messages, temperature):
    for _ in range(3):
        try:
            print(f"--> Sending to Mistral @ T={temperature}")
            resp = client.chat.complete(
                model=MODEL_NAME,
                temperature=temperature,
                messages=messages
            )
            answer = resp.choices[0].message.content.strip()
            print("=== MODEL RESPONSE ===")
            print(answer)
            print("======================\n")

            return answer
        except Exception as e:
            print(f"Error: {e}, retrying...")
            time.sleep(PAUSE_SECONDS)
    return "ERROR"

def mistral_complete(messages, temperature: float, n: int = 1, max_retries: int = 5):
    """
    Call Mistral repeatedly in batches of up to 10 to reach n completions.
    """
    max_per_call = 10
    collected = []
    delay = PAUSE_SECONDS

    while len(collected) < n:
        need = min(max_per_call, n - len(collected))
        for attempt in range(1, max_retries + 1):
            try:
                print(f"--> Mistral call @ T={temperature}, batch={need}, collected={len(collected)}/{n}")
                resp = client.chat.complete(
                    model=MODEL_NAME,
                    temperature=temperature,
                    n=need,
                    messages=messages
                )
                for ch in resp.choices:
                    collected.append(ch.message.content.strip())
                break  # success → move to next batch
            except Exception as e:
                print(f"[mistral_complete] Error: {e} (attempt {attempt}/{max_retries})")
                time.sleep(delay)
                delay = min(delay * 2, 8.0)
        else:
            # after max_retries
            collected.extend(["ERROR"] * need)

    return collected[:n]

# =========================
# RUNNERS
# =========================
def _load_done_index_from_jsonl(path: Path):
    """
    Reads an existing JSONL file and returns a set of keys for already-completed records.
    Key schema: (user_id, template_set, style, fairness, temperature)
    """
    done = set()
    if not path.exists():
        return done
    with path.open("r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            try:
                obj = json.loads(line)
                key = (
                    int(obj.get("user_id")),
                    obj.get("template_set"),
                    obj.get("style"),
                    obj.get("fairness"),
                    float(obj.get("temperature", 0.0)),
                )
                done.add(key)
            except Exception:
                # skip malformed lines, don't crash resume
                continue
    return done


def run_normal_t0(df_users, template_sets=("A","B","C")):
    """
    Normal run @ T=0.0 for all sets (A,B,C) and styles (zero_shot, few_shot, zero_shot_cot, few_shot_cot).
    Writes each record to JSONL immediately and supports resume by skipping already written items.
    """
    out_path = OUTPUT_NORMAL / "normal_t0.jsonl"
    done = _load_done_index_from_jsonl(out_path)

    # Build all tasks, skipping those already completed
    tasks = []
    for _, row in df_users.iterrows():
        uid = int(row.user_id)
        history_text = format_history(row.get("sample_random", []))
        for set_name in template_sets:
            for style in TEMPLATE_SETS[set_name].keys():
                for fair in FAIRNESS_LEVELS:
                    key = (uid, set_name, style, fair, 0.0)
                    if key in done:
                        continue
                    tasks.append((uid, set_name, style, fair, history_text))

    if not tasks:
        print(f"[Normal] Nothing to do. All results already present in {out_path}")
        return

    # Open for append; write/flush each result ASAP
    with out_path.open("a", encoding="utf-8") as f_out:
        with ThreadPoolExecutor(max_workers=MAX_WORKERS) as pool:
            futures = []
            for uid, set_name, style, fair, history_text in tasks:
                print(f"Running NORMAL: user={uid}, set={set_name}, style={style}, fairness={fair}")
                msgs = build_messages(set_name, style, fair, TOP_K, history_text)
                futures.append(
                        pool.submit(
                        lambda u=uid, sn=set_name, st=style, fk=fair, m=msgs:
                        (u, sn, st, fk, 0.0, chat_once(m, 0.0), m)
                    )
                )


            for fut in tqdm(as_completed(futures), total=len(futures), desc="Normal T=0.0"):
                try:
                    user_id, set_name, style, fair, temp, answer, msgs = fut.result()
                except Exception as e:
                    print(f"[Normal] Future failed: {e}")
                    continue

                record = {
                    "user_id": user_id,
                    "template_set": set_name,
                    "style": style,
                    "fairness": fair,
                    "temperature": temp,
                    "messages": msgs,
                    "response": answer
                }

                # Write ASAP + force flush so you can resume safely after interruptions
                f_out.write(json.dumps(record, ensure_ascii=False) + "\n")
                f_out.flush()
                os.fsync(f_out.fileno())


def run_sc(df_users, temps=(0.2,0.5,0.7,1.0), template_sets=("A","B","C"), N=20):
    cot_styles = ["zero_shot_cot", "few_shot_cot"]

    for _, row in df_users.iterrows():
        uid = int(row.user_id)
        history_text = format_history(row.get("sample_random", []))

        for set_name in template_sets:
            for t in temps:
                base = OUTPUT_SC / f"user_{uid}" / set_name / f"temp_{t}"
                base.mkdir(parents=True, exist_ok=True)

                for style in cot_styles:
                    for fair in FAIRNESS_LEVELS:
                        out_file = base / f"{style}__{fair}.json"
                        if out_file.exists():
                            print(f"[Skip] {out_file} already exists")
                            continue

                        print(f"Running SC: user={uid}, set={set_name}, style={style}, fairness={fair}, temp={t}, N={N}")
                        msgs = build_messages(set_name, style, fair, TOP_K, history_text)

                        # ---- ONE API call returns N completions ----
                        completions = mistral_complete(msgs, temperature=t, n=N)

                        if not completions:
                            samples = [{"sample_id": i + 1, "response": "ERROR: empty"} for i in range(N)]
                        else:
                            samples = [{"sample_id": i + 1, "response": txt} for i, txt in enumerate(completions)]

                        record = {
                            "user_id": uid,
                            "template_set": set_name,
                            "style": style,
                            "fairness": fair,
                            "temperature": t,
                            "messages": msgs,
                            "samples": samples
                        }
                        with out_file.open("w", encoding="utf-8") as f:
                            json.dump(record, f, ensure_ascii=False, indent=2)
                            f.flush()
                            os.fsync(f.fileno())

# =========================
# NORMAL RUNS FOR TEMPLATE B @ EXTRA TEMPS
# =========================
NORMAL_EXTRA_TEMPS = [0.2, 0.5, 0.7, 1.0]

def run_normal_templateB(
    df_users,
    temps=NORMAL_EXTRA_TEMPS,
    styles=("zero_shot", "few_shot", "zero_shot_cot", "few_shot_cot")
):
    """
    For Template Set C only, run normal single-completion calls at the given temperatures
    across the provided styles. Writes one JSONL per temperature and supports resume.
    """
    for temp in temps:
        # nice filename like normal_B_t0p2.jsonl, normal_B_t1.jsonl, etc.
        ttag = str(temp).replace(".", "p")
        out_path = OUTPUT_NORMAL / f"normal_C_t{ttag}.jsonl"
        done = _load_done_index_from_jsonl(out_path)

        # Build tasks, skipping already-completed (resume-friendly)
        tasks = []
        for _, row in df_users.iterrows():
            uid = int(row.user_id)
            history_text = format_history(row.get("sample_random", []))
            for fair in FAIRNESS_LEVELS:
                for style in styles:
                    key = (uid, "B", style, fair, float(temp))
                    if key in done:
                        continue
                    tasks.append((uid, "B", style, fair, history_text, temp))

        if not tasks:
            print(f"[Normal-C@T={temp}] Nothing to do. All results already present in {out_path}")
            continue

        with out_path.open("a", encoding="utf-8") as f_out:
            with ThreadPoolExecutor(max_workers=MAX_WORKERS) as pool:
                futures = []
                for uid, set_name, style, fair, history_text, t in tasks:
                    print(f"Running NORMAL-B: user={uid}, style={style}, fairness={fair}, T={t}")
                    msgs = build_messages(set_name, style, fair, TOP_K, history_text)
                    futures.append(
                        pool.submit(
                            lambda u=uid, sn=set_name, st=style, fk=fair, m=msgs, tt=t:
                                (u, sn, st, fk, tt, chat_once(m, tt), m)
                        )
                    )

                for fut in tqdm(as_completed(futures), total=len(futures), desc=f"Normal B T={temp}"):
                    try:
                        user_id, set_name, style, fair, temp_used, answer, msgs = fut.result()
                    except Exception as e:
                        print(f"[Normal-C@T={temp}] Future failed: {e}")
                        continue

                    record = {
                        "user_id": user_id,
                        "template_set": set_name,  # "B"
                        "style": style,
                        "fairness": fair,
                        "temperature": float(temp_used),
                        "messages": msgs,
                        "response": answer
                    }
                    f_out.write(json.dumps(record, ensure_ascii=False) + "\n")
                    f_out.flush()
                    os.fsync(f_out.fileno())



# =========================
# SELECT USERS & RUN
# =========================
# Pick 30 users deterministically
all_user_ids = df_filtered_user_data["user_id"].dropna().astype(int).unique().tolist()
random.seed(42)
picked = sorted(random.sample(all_user_ids, k=min(100, len(all_user_ids))))
print("Selected 100 user IDs:", picked)

users_to_run = df_filtered_user_data[df_filtered_user_data["user_id"].isin(picked)].copy()


print("Running for users:", users_to_run["user_id"].unique().tolist())
#run_normal_templateB(users_to_run, temps=[0.2, 0.5, 0.7, 1.0],
 #                    styles=("zero_shot", "few_shot", "zero_shot_cot", "few_shot_cot"))

# 1) Normal run @ T=0.0 (single file)
run_normal_t0(users_to_run, template_sets=("A","B","C"))

#run_normal_templateC(users_to_run, temps=[0.2, 0.5, 0.7, 1.0],styles=("zero_shot", "few_shot", "zero_shot_cot", "few_shot_cot"))

# 2) Self-consistency CoT runs (per-temp → per-user → per (style×fairness) file)
#run_sc(users_to_run, temps=SC_TEMPS, template_sets=("B"))


print("✓ Done.")


In [ ]:
pip install mistralai